In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, count, when

In [2]:
spark = SparkSession.builder.appName('Features').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 09:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [27]:
shot_events = events.filter(col('type')=='Shot')
shot_events.count()

61608

In [ ]:
to_drop = []
l = shot_events.count()
for column in shot_events.columns:
    if shot_events.filter(col(column).isNull()).count() == l:
        to_drop.append(column)
    print(column)

In [32]:
cols = [c for c in shot_events.columns if c not in to_drop]

In [9]:
# Reorder
SHOT_COLUMNS = ['id','period','duration','location','player_id','position', # Player info
                'play_pattern','shot_body_part','shot_technique','shot_type',        # Shot info
                'related_events', 'shot_freeze_frame', 'shot_key_pass_id', 'shot_end_location', # Complicated info
                'under_pressure','shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble', # Boolean
                'shot_statsbomb_xg','shot_outcome'# Target
                ]

In [28]:
shot_events = shot_events.select(SHOT_COLUMNS)

In [11]:
shot_events.show()

+--------------------+------+--------+-------------+---------+--------------------+--------------+--------------+--------------+---------+--------------------+--------------------+--------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+
|                  id|period|duration|     location|player_id|            position|  play_pattern|shot_body_part|shot_technique|shot_type|      related_events|   shot_freeze_frame|    shot_key_pass_id| shot_end_location|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|
+--------------------+------+--------+-------------+---------+--------------------+--------------+--------------+--------------+---------+--------------------+--------------------+--------------------+------------------+--------------+---------------+---------------+---------------+--------------+

In [ ]:
# Unfold dummies # drop other # dummies -1
shot_events.groupBy('play_pattern').count().orderBy('count',ascending=False).show()

+--------------+-----+
|  play_pattern|count|
+--------------+-----+
|  Regular Play|20380|
|From Free Kick|12822|
| From Throw In|10650|
|   From Corner| 9833|
|  From Counter| 2762|
|From Goal Kick| 2390|
|   From Keeper| 1042|
|         Other| 1027|
| From Kick Off|  702|
+--------------+-----+



In [29]:
shot_events = shot_events.withColumn('other_pp',when((col('play_pattern')=='Other'),1).otherwise(0)) \
    .withColumn('from_fk',when((col('play_pattern')=='From Free Kick'),1).otherwise(0)) \
    .withColumn('from_ti',when((col('play_pattern')=='From Throw In'),1).otherwise(0)) \
    .withColumn('from_corner',when((col('play_pattern')=='From Corner'),1).otherwise(0)) \
    .withColumn('from_counter',when((col('play_pattern')=='From Counter'),1).otherwise(0)) \
    .withColumn('from_gk',when((col('play_pattern')=='From Goal Kick'),1).otherwise(0)) \
    .withColumn('from_keeper',when((col('play_pattern')=='From Keeper'),1).otherwise(0)) \
    .withColumn('ko',when((col('play_pattern')=='From Kick Off'),1).otherwise(0)).drop('play_pattern')

In [52]:
# Head columns to be added # drop other
shot_events.groupBy('shot_body_part').count().orderBy('count',ascending=False).show()

+--------------+-----+
|shot_body_part|count|
+--------------+-----+
|    Right Foot|31623|
|     Left Foot|19880|
|          Head| 9922|
|         Other|  183|
+--------------+-----+



In [30]:
shot_events = shot_events.withColumn('header',when((col('shot_body_part')=='Head'),1).otherwise(0)).drop('shot_body_part')

In [24]:
# Shot type
shot_events.groupBy('shot_type').count().orderBy('count',ascending=False).show()

+---------+-----+
|shot_type|count|
+---------+-----+
|Open Play|57719|
|Free Kick| 2932|
|  Penalty|  938|
|   Corner|   19|
+---------+-----+



In [31]:
shot_events =shot_events.withColumn('corner_type',when((col('shot_type')=='Corner'),1).otherwise(0)) \
.withColumn('fk_type',when((col('shot_type')=='Free Kick'),1).otherwise(0)) \
.withColumn('pk_type',when((col('shot_type')=='Penalty'),1).otherwise(0)) \
.drop('shot_type')

In [54]:
# Shot technique
shot_events.groupBy('shot_technique').count().orderBy('count',ascending=False).show()

+--------------+-----+
|shot_technique|count|
+--------------+-----+
|        Normal|47396|
|   Half Volley| 8711|
|        Volley| 4091|
|           Lob|  578|
| Overhead Kick|  351|
|      Backheel|  242|
| Diving Header|  239|
+--------------+-----+



In [32]:
shot_events =shot_events.withColumn('half_volley_technique',when((col('shot_technique')=='Half Volley'),1).otherwise(0)) \
.withColumn('volley_technique',when((col('shot_technique')=='Volley'),1).otherwise(0)) \
.withColumn('lob_technique',when((col('shot_technique')=='Lob'),1).otherwise(0)) \
.withColumn('overhead_technique',when((col('shot_technique')=='Overhead Kick'),1).otherwise(0)) \
.withColumn('backheel_technique',when((col('shot_technique')=='Backheel'),1).otherwise(0)) \
.withColumn('diving_h_technique',when((col('shot_technique')=='Diving Header'),1).otherwise(0)) \
.drop('shot_technique')

In [33]:
shot_events.show()

+--------------------+------+--------+-------------+---------+--------------------+--------------------+--------------------+--------------------+------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+--------+-------+-------+-----------+------------+-------+-----------+---+------+-----------+-------+-------+---------------------+----------------+-------------+------------------+------------------+------------------+
|                  id|period|duration|     location|player_id|            position|      related_events|   shot_freeze_frame|    shot_key_pass_id| shot_end_location|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|other_pp|from_fk|from_ti|from_corner|from_counter|from_gk|from_keeper| ko|header|corner_type|fk_type|pk_type|half_volley_technique|volley_technique|lob_technique|overhead_technique|backheel_